In [ ]:
import sys
sys.path.append('../../caml-mimic/')
# put mimic3 related files in MIMIC_3_DIR
from constants import MIMIC_4_SAVE_DIR
import numpy as np
import pandas as pd
from icd_50 import codes_50

In [ ]:
from tqdm import tqdm
import csv

note_dict = {}
others_dict = {}

notes_file = '%s/NOTEEVENTS.csv' % (MIMIC_3_DIR)
with open(notes_file, 'r') as csvfile:
    notereader = csv.reader(csvfile)
    next(notereader)
    i = 0
    for line in tqdm(notereader):
        subject_id = int(line[1])
        hadm_id = str(line[2])
        category = str(line[6])
        note = line[10]
        if (subject_id, hadm_id) not in note_dict:
            note_dict[(subject_id, hadm_id)] = []
            others_dict[(subject_id, hadm_id)] = []
        if category == "Discharge summary":
            note_dict[(subject_id, hadm_id)].append(note)
        else:
            others_dict[(subject_id, hadm_id)].append(category + ": " + note)
        i += 1

In [ ]:
df_codes = pd.read_csv('%s/ALL_CODES_filtered.csv' % MIMIC_3_DIR, index_col=None)
print(len(set(df_codes['HADM_ID'])))

In [ ]:
def split_note_to_paragraph(note):
    result = {}
    now_paragraph = 'other'
    for line in note.split('\n'):
        if line.find(":") >= 0:
            now_paragraph = line[0:line.find(":")]
            text = line[line.find(":") + 1:]
            if not now_paragraph in result:
                result[now_paragraph] = []
            result[now_paragraph].append(text.strip())
        elif not line.strip():
            now_paragraph = 'other'
        else:
            if not now_paragraph in result:
                result[now_paragraph] = []
            result[now_paragraph].append(line.strip())
    for paragraph in result:
        result[paragraph] = " ".join(result[paragraph])
    return result

# split_note_to_paragraph(text_list[1])

In [ ]:
# p_count = {}
# for idx, text in tqdm(enumerate(text_list)):
#     result = split_note_to_paragraph(text)
#     for key in result.keys():
#         if not key in p_count:
#             p_count[key] = 0
#         p_count[key] += 1
# print(len(p_count))
# p_count_sort = {k: v for k, v in sorted(p_count.items(), key=lambda item: -item[1])}
# list(p_count_sort.items())[0:100]

In [ ]:
from tqdm import trange

icd_dict = {}
for i in trange(df_codes.shape[0]):
    subject_id = df_codes['SUBJECT_ID'][i]
    hadm_id = str(df_codes['HADM_ID'][i])
    code = df_codes['ICD9_CODE'][i]
    if (subject_id, hadm_id) not in icd_dict:
        icd_dict[(subject_id, hadm_id)] = []
    icd_dict[(subject_id, hadm_id)].append(code)

In [ ]:
# for i in trange(df_notes.shape[0]):
#     subject_id = df_notes['SUBJECT_ID'][i]
#     hadm_id = df_notes['HADM_ID'][i]
#     text = df_notes['TEXT'][i]
#     if (subject_id, hadm_id) in note_dict:
#         note_dict[(subject_id, hadm_id)].append(text)

In [ ]:
for base_name in ['mimic3', 'mimic3-50']:
    train_name = '%s_train.json' % (base_name)
    dev_name = '%s_dev.json' % (base_name)
    test_name = '%s_test.json' % (base_name)

    hadm_ids = {}

    #read in train, dev, test splits
    for splt in ['train', 'dev', 'test']:
        hadm_ids[splt] = set()
        if base_name == "mimic3":
            base = "full"
        if base_name == "mimic3-50":
            base = "50"
        with open('%s/%s_%s_hadm_ids.csv' % (MIMIC_3_DIR, splt, base), 'r') as f:
            for line in f:
                hadm_ids[splt].add(line.rstrip())

    train_list = []
    dev_list = []
    test_list = []

    for key in tqdm(icd_dict.keys()):
        subject_id = str(int(key[0]))
        hadm_id = str(int(key[1]))
        icd = ";".join([str(c) for c in icd_dict[key]])
        if base_name == "mimic-50":
            filtered_codes = set(icd_dict[key]).intersection(set(codes_50))
            if len(filtered_codes) > 0:
                icd = ";".join([str(c) for c in filtered_codes])
            else:
                continue

        text = "\t".join([str(c) for c in note_dict[key]])
        additional_text = "\t".join([str(c) for c in others_dict[key]])
        row = {'subject_id':subject_id,
               'hadm_id':hadm_id,
               'LABELS':icd,
               'TEXT':text,
               'Addition':additional_text}

        if hadm_id in hadm_ids['train']:
            train_list.append(row)
        elif hadm_id in hadm_ids['dev']:
            dev_list.append(row)
        elif hadm_id in hadm_ids['test']:
            test_list.append(row)
        else:
            #print(key)
            pass

    print(len(train_list), len(dev_list), len(test_list))
    import json
    with open(train_name, "w") as f:
        json.dump(train_list, f, indent=4)
    with open(dev_name, "w") as f:
        json.dump(dev_list, f, indent=4)
    with open(test_name, "w") as f:
        json.dump(test_list, f, indent=4)

---

In [1]:
import pandas as pd

import sys
sys.path.append('../../caml-mimic/')
from constants import MIMIC_4_SAVE_DIR
import json

In [2]:
def make_json(splt):
    filename = '%s/%s_%s.csv' % (MIMIC_4_SAVE_DIR, splt, str(50))
    df = pd.read_csv(filename)
    res_list = list(df.iloc[:,:-1].T.to_dict().values())
    with open('%s/%s_%s.json' % (MIMIC_4_SAVE_DIR, splt, str(50)), "w") as f:
        json.dump(res_list, f, indent=4)
    return res_list

In [10]:
res = make_json("test")